In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_path = "Images"
img_size = (200, 200)
batch_size = 32

def load_data(dataset_path, img_size, batch_size):
    transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),
    ])
    dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=transform)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, len(dataset.classes)

train_loader, val_loader, num_classes = load_data(dataset_path, img_size, batch_size)


class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 25 * 25, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    model.to(device)
    loss_history = []
    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        epoch_loss = running_loss / len(train_loader)
        loss_history.append(epoch_loss)
        print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training Finished. Time taken: {training_time:.2f} seconds")
    return loss_history, training_time

optimizer_configs = {
    "SGD": [
        {"name": "SGD_momentum_0.5", "class": optim.SGD, "params": {"lr": 0.01, "momentum": 0.5}},
        {"name": "SGD_momentum_0.9", "class": optim.SGD, "params": {"lr": 0.01, "momentum": 0.9}},
        {"name": "SGD_momentum_0.95", "class": optim.SGD, "params": {"lr": 0.01, "momentum": 0.95}}
    ],
    "Adam": [
        {"name": "Adam_default", "class": optim.Adam, "params": {"lr": 0.001, "betas": (0.9, 0.999)}},
        {"name": "Adam_beta1_0.8", "class": optim.Adam, "params": {"lr": 0.001, "betas": (0.8, 0.99)}},
        {"name": "Adam_beta2_0.9", "class": optim.Adam, "params": {"lr": 0.001, "betas": (0.95, 0.9)}}
    ],
    "RMSprop": [
        {"name": "RMSprop_default", "class": optim.RMSprop, "params": {"lr": 0.001, "momentum": 0.9}}
    ],
    "Adagrad": [
        {"name": "Adagrad_default", "class": optim.Adagrad, "params": {"lr": 0.01}}
    ]
}

loss_histories = {}
training_times = {}

for opt_group, configs in optimizer_configs.items():
    for config in configs:
        print(f"\nTraining with {config['name']}")
        model = CNN(num_classes)
        criterion = nn.CrossEntropyLoss()
        optimizer = config["class"](model.parameters(), **config["params"])
        loss_history, training_time = train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)
        loss_histories[config["name"]] = loss_history
        training_times[config["name"]] = training_time

def plot_loss_curves(loss_histories, title, keys, epochs=10):
    plt.figure(figsize=(10, 6))
    for key in keys:
        plt.plot(range(1, epochs + 1), loss_histories[key], label=key)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig(title)
    plt.show()


plot_loss_curves(loss_histories, "Loss Curves for SGD with Different Momentum Values",
                 ["SGD_momentum_0.5", "SGD_momentum_0.9", "SGD_momentum_0.95"])


plot_loss_curves(loss_histories, "Loss Curves for Adam with Different Beta Values",
                 ["Adam_default", "Adam_beta1_0.8", "Adam_beta2_0.9"])


best_configs = ["SGD_momentum_0.9", "Adam_default", "RMSprop_default", "Adagrad_default"]

plot_loss_curves(loss_histories, "Loss Curves for Best Configurations of Each Optimizer", best_configs)


print("\nTraining Times Summary:")
print("{:<20} {:<15}".format("Optimizer", "Time (seconds)"))
print("-" * 35)
for name, t in training_times.items():
    print("{:<20} {:<15.2f}".format(name, t))


Training with SGD_momentum_0.5
Epoch 1, Loss: 3.0441
Epoch 2, Loss: 3.0361
Epoch 3, Loss: 3.0228
Epoch 4, Loss: 2.9900
Epoch 5, Loss: 2.8874
Epoch 6, Loss: 2.7605
Epoch 7, Loss: 2.6620
Epoch 8, Loss: 2.5013
Epoch 9, Loss: 2.3217
Epoch 10, Loss: 2.0962
Training Finished. Time taken: 1402.44 seconds

Training with SGD_momentum_0.9
Epoch 1, Loss: 3.0429
Epoch 2, Loss: 2.9819
Epoch 3, Loss: 2.7980
Epoch 4, Loss: 2.4072
Epoch 5, Loss: 2.1596
Epoch 6, Loss: 1.7645
Epoch 7, Loss: 1.2951
Epoch 8, Loss: 0.8999
Epoch 9, Loss: 0.5303
Epoch 10, Loss: 0.3829
Training Finished. Time taken: 113.82 seconds

Training with SGD_momentum_0.95
Epoch 1, Loss: 3.0455
Epoch 2, Loss: 2.9813
Epoch 3, Loss: 2.8955
Epoch 4, Loss: 2.4198
Epoch 5, Loss: 1.9822
Epoch 6, Loss: 1.6330
Epoch 7, Loss: 1.1495
Epoch 8, Loss: 0.9501
Epoch 9, Loss: 0.6319
Epoch 10, Loss: 0.4969
Training Finished. Time taken: 114.09 seconds

Training with Adam_default
Epoch 1, Loss: 2.9628
Epoch 2, Loss: 2.5001
Epoch 3, Loss: 1.8350
Epoch 4